<a href="https://colab.research.google.com/github/binaypanda/Fruit-In-Sight/blob/main/neem_azadirachtin_sota_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GoogLeNet

In [ ]:
!nvidia-smi

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import numpy as np
import random
from PIL import Image
from skimage import io
from skimage.color import rgb2gray

In [ ]:
class train(Dataset):
    def __init__(self, train_csv_file,root_dir, transform=None):
        self.train_annotations = pd.read_csv(train_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.train_annotations)

    def __getitem__(self, index):
        # load a regular image
        image_path = os.path.join(self.root_dir, self.train_annotations.iloc[index, 0])
        y_label = int(self.train_annotations.iloc[index, 1])

        image = io.imread(image_path)

        # for greyscale images
        if len(image.shape) == 2:
            # Duplicate single channel across all three channels
            image = np.repeat(image[:, :, np.newaxis], 3, axis=2)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)


class test(Dataset):
    def __init__(self, test_csv_file, root_dir, transform=None):
        self.test_annotations = pd.read_csv(test_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.test_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.test_annotations.iloc[index, 0])
        y_label = int(self.test_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

class val(Dataset):
    def __init__(self, val_csv_file, root_dir, transform=None):
        self.val_annotations = pd.read_csv(val_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.val_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.val_annotations.iloc[index, 0])
        y_label = int(self.val_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyperparameters
in_channel = 3
learning_rate = 1e-3
batch_size = 2
num_epochs = 100

#transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.Resize((640,640))
])

train_dataset = train(train_csv_file='/home/neem/a_train.csv', root_dir='/home/neem/a_dataset', transform=transform)

test_dataset = test(test_csv_file='/home/neem/a_test.csv', root_dir='/home/neem/a_dataset', transform=transform)

val_dataset = val(val_csv_file='/home/neem/a_val.csv', root_dir='/home/neem/a_dataset', transform=transform)

In [ ]:
print(len(train_dataset))

In [ ]:
print(len(test_dataset))

In [ ]:
print(len(val_dataset))

In [ ]:
#train_set, test_set, val_set = torch.utils.data.random_split(dataset, [2834, 354, 354], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for items in train_loader:
    print(items)

In [ ]:
len(train_loader)

In [ ]:
#model
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Train network
t_loss = []
v_loss = []
for epoch in range(num_epochs):
  losses = []
  model.train()
  for batch_idx, (data, targets) in enumerate(train_loader):
    #data to cuda if possible
    data = data.to(device = device)
    targets = targets.to(device = device)

    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    losses.append(loss.item())

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss = sum(losses)/len(losses)
  #print(f'Training loss at epoch {epoch} is {train_loss}')

  model.eval()
  loss = 0
  with torch.no_grad():
    for x, y in val_loader:
      x, y = x.to(device=device), y.to(device=device)
      out = model(x)
      loss += criterion(out, y)
  val_loss = loss/len(val_loader)
  print(f'Training and Validation loss at epoch {epoch} is {train_loss} and {val_loss}')
  t_loss.append(train_loss)
  v_loss.append(val_loss.item())

In [ ]:
import matplotlib.pyplot as plt
plt1 = plt.figure(figsize=(25,15))

x_axis = []
for i in range(num_epochs):
  x_axis.append(i)
plt.plot(x_axis, t_loss, 'g', label = 'training loss')
plt.plot(x_axis, v_loss, 'r', label = 'validation loss')
plt.legend(['training loss', 'validation loss'])
plt.title('training loss vs validation loss for GoogleNet')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()
  Y = np.array([0])
  Y_pred = np.array([0])
  f1 = 0
  i=0
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)
      y = y.to(device)
      Y = np.append(Y,y.cpu())
      i += 1
      print('Y after appendingt ', i, ' times : ', Y)
      scores = model(x)
      _, predictions = scores.max(1)
      print(list(predictions))
      Y_pred = np.append(Y_pred, predictions.cpu())
      print('Y_pred after appending: ', Y_pred)
      num_correct += (predictions == y).sum()
      num_samples = predictions.size(0)
    print(f'got accuracy {float(num_correct)/float(num_samples*100)}')
  Y = torch.IntTensor(Y)
  print('Y is : ', Y)
  Y_pred = torch.IntTensor(Y_pred)
  print('Y_pred is: ', Y_pred)
  f1 = f1_score(Y, Y_pred)
  #print('f1 score: ', f1)
  #print('precision, recall, f1 score are (ignore None): ', precision_recall_fscore_support(Y, Y_pred, average = 'binary'))
  print(classification_report(Y, Y_pred))
  model.train()

In [ ]:
torch.save(model.state_dict(), '/home/neem/weights/weights_googlenet.pt')

In [ ]:
print('Checking accuracy on val set:')
check_accuracy(val_loader, model)

In [ ]:
print('Checking accuracy on test set:')
check_accuracy(test_loader, model)

In [ ]:
print('Checking accuracy on training set:')
check_accuracy(train_loader, model)

# Inception v3

In [ ]:
!nvidia-smi

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import numpy as np
import random
from PIL import Image
from skimage import io
from skimage.color import rgb2gray

In [ ]:
class train(Dataset):
    def __init__(self, train_csv_file,root_dir, transform=None):
        self.train_annotations = pd.read_csv(train_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.train_annotations)

    def __getitem__(self, index):
        # load a regular image
        image_path = os.path.join(self.root_dir, self.train_annotations.iloc[index, 0])
        y_label = int(self.train_annotations.iloc[index, 1])

        image = io.imread(image_path)

        # for greyscale images
        if len(image.shape) == 2:
            # Duplicate single channel across all three channels
            image = np.repeat(image[:, :, np.newaxis], 3, axis=2)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)


class test(Dataset):
    def __init__(self, test_csv_file, root_dir, transform=None):
        self.test_annotations = pd.read_csv(test_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.test_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.test_annotations.iloc[index, 0])
        y_label = int(self.test_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

class val(Dataset):
    def __init__(self, val_csv_file, root_dir, transform=None):
        self.val_annotations = pd.read_csv(val_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.val_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.val_annotations.iloc[index, 0])
        y_label = int(self.val_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyperparameters
in_channel = 3
learning_rate = 1e-3
batch_size = 2
num_epochs = 100

#transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.Resize((640,640))
])

train_dataset = train(train_csv_file='/home/neem/a_train.csv', root_dir='/home/neem/a_dataset', transform=transform)

test_dataset = test(test_csv_file='/home/neem/a_test.csv', root_dir='/home/neem/a_dataset', transform=transform)

val_dataset = val(val_csv_file='/home/neem/a_val.csv', root_dir='/home/neem/a_dataset', transform=transform)

In [ ]:
print(len(train_dataset))

In [ ]:
print(len(test_dataset))

In [ ]:
print(len(val_dataset))

In [ ]:
#train_set, test_set, val_set = torch.utils.data.random_split(dataset, [2834, 354, 354], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for items in train_loader:
    print(items)

In [ ]:
len(train_loader)

In [ ]:
#model
model = torchvision.models.inception_v3(pretrained=True)
model.to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Train network
t_loss = []
v_loss = []
for epoch in range(num_epochs):
  losses = []
  model.train()
  for batch_idx, (data, targets) in enumerate(train_loader):
    #data to cuda if possible
    data = data.to(device = device)
    targets = targets.to(device = device)

    #forward
    scores, y = model(data)
    loss = criterion(scores, targets)

    losses.append(loss.item())

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  train_loss = sum(losses)/len(losses)
  #print(f'Training loss at epoch {epoch} is {train_loss}')

  model.eval()
  loss = 0
  with torch.no_grad():
    for x, y in val_loader:
      x, y = x.to(device=device), y.to(device=device)
      out = model(x)
      loss += criterion(out, y)
  val_loss = loss/len(val_loader)
  print(f'Training and Validation loss at epoch {epoch} is {train_loss} and {val_loss}')
  t_loss.append(train_loss)
  v_loss.append(val_loss.item())

In [ ]:
import matplotlib.pyplot as plt
plt1 = plt.figure(figsize=(25,15))

x_axis = []
for i in range(num_epochs):
  x_axis.append(i)
plt.plot(x_axis, t_loss, 'g', label = 'training loss')
plt.plot(x_axis, v_loss, 'r', label = 'validation loss')
plt.legend(['training loss', 'validation loss'])
plt.title('training loss vs validation loss for InceptionNet')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()
  Y = np.array([0])
  Y_pred = np.array([0])
  f1 = 0
  i=0
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)
      y = y.to(device)
      Y = np.append(Y,y.cpu())
      i += 1
      print('Y after appendingt ', i, ' times : ', Y)
      scores = model(x)
      _, predictions = scores.max(1)
      print(list(predictions))
      Y_pred = np.append(Y_pred, predictions.cpu())
      print('Y_pred after appending: ', Y_pred)
      num_correct += (predictions == y).sum()
      num_samples = predictions.size(0)
    print(f'got accuracy {float(num_correct)/float(num_samples*100)}')
  Y = torch.IntTensor(Y)
  print('Y is : ', Y)
  Y_pred = torch.IntTensor(Y_pred)
  print('Y_pred is: ', Y_pred)
  f1 = f1_score(Y, Y_pred)
  #print('f1 score: ', f1)
  #print('precision, recall, f1 score are (ignore None): ', precision_recall_fscore_support(Y, Y_pred, average = 'binary'))
  print(classification_report(Y, Y_pred))
  model.train()

In [ ]:
torch.save(model.state_dict(), '/home/neem/weights/weights_Inceptionv3.pt')

In [ ]:
print('Checking accuracy on val set:')
check_accuracy(val_loader, model)

In [ ]:
print('Checking accuracy on test set:')
check_accuracy(test_loader, model)

In [ ]:
print('Checking accuracy on training set:')
check_accuracy(train_loader, model)

# EfficientNetB0

In [ ]:
!nvidia-smi

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import numpy as np
import random
from PIL import Image
from skimage import io
from skimage.color import rgb2gray

In [ ]:
class train(Dataset):
    def __init__(self, train_csv_file,root_dir, transform=None):
        self.train_annotations = pd.read_csv(train_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.train_annotations)

    def __getitem__(self, index):
        # load a regular image
        image_path = os.path.join(self.root_dir, self.train_annotations.iloc[index, 0])
        y_label = int(self.train_annotations.iloc[index, 1])

        image = io.imread(image_path)

        # for greyscale images
        if len(image.shape) == 2:
            # Duplicate single channel across all three channels
            image = np.repeat(image[:, :, np.newaxis], 3, axis=2)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)


class test(Dataset):
    def __init__(self, test_csv_file, root_dir, transform=None):
        self.test_annotations = pd.read_csv(test_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.test_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.test_annotations.iloc[index, 0])
        y_label = int(self.test_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

class val(Dataset):
    def __init__(self, val_csv_file, root_dir, transform=None):
        self.val_annotations = pd.read_csv(val_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.val_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.val_annotations.iloc[index, 0])
        y_label = int(self.val_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyperparameters
in_channel = 3
learning_rate = 1e-3
batch_size = 2
num_epochs = 100

#transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.Resize((640,640))
])

train_dataset = train(train_csv_file='/home/neem/a_train.csv', root_dir='/home/neem/a_dataset', transform=transform)

test_dataset = test(test_csv_file='/home/neem/a_test.csv', root_dir='/home/neem/a_dataset', transform=transform)

val_dataset = val(val_csv_file='/home/neem/a_val.csv', root_dir='/home/neem/a_dataset', transform=transform)

In [ ]:
print(len(train_dataset))

In [ ]:
print(len(test_dataset))

In [ ]:
print(len(val_dataset))

In [ ]:
#train_set, test_set, val_set = torch.utils.data.random_split(dataset, [2834, 354, 354], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for items in train_loader:
    print(items)

In [ ]:
len(train_loader)

In [ ]:
#model
model = torchvision.models.efficientnet_b0(pretrained=True)
model.to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Train network
t_loss = []
v_loss = []
for epoch in range(num_epochs):
  losses = []
  model.train()
  for batch_idx, (data, targets) in enumerate(train_loader):
    #data to cuda if possible
    data = data.to(device = device)
    targets = targets.to(device = device)

    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    losses.append(loss.item())

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss = sum(losses)/len(losses)
  #print(f'Training loss at epoch {epoch} is {train_loss}')

  model.eval()
  loss = 0
  with torch.no_grad():
    for x, y in val_loader:
      x, y = x.to(device=device), y.to(device=device)
      out = model(x)
      loss += criterion(out, y)
  val_loss = loss/len(val_loader)
  print(f'Training and Validation loss at epoch {epoch} is {train_loss} and {val_loss}')
  t_loss.append(train_loss)
  v_loss.append(val_loss.item())

In [ ]:
import matplotlib.pyplot as plt
plt1 = plt.figure(figsize=(25,15))

x_axis = []
for i in range(num_epochs):
  x_axis.append(i)
plt.plot(x_axis, t_loss, 'g', label = 'training loss')
plt.plot(x_axis, v_loss, 'r', label = 'validation loss')
plt.legend(['training loss', 'validation loss'])
plt.title('training loss vs validation loss for EfficientNetB0')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()
  Y = np.array([0])
  Y_pred = np.array([0])
  f1 = 0
  i=0
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)
      y = y.to(device)
      Y = np.append(Y,y.cpu())
      i += 1
      print('Y after appendingt ', i, ' times : ', Y)
      scores = model(x)
      _, predictions = scores.max(1)
      print(list(predictions))
      Y_pred = np.append(Y_pred, predictions.cpu())
      print('Y_pred after appending: ', Y_pred)
      num_correct += (predictions == y).sum()
      num_samples = predictions.size(0)
    print(f'got accuracy {float(num_correct)/float(num_samples*100)}')
  Y = torch.IntTensor(Y)
  print('Y is : ', Y)
  Y_pred = torch.IntTensor(Y_pred)
  print('Y_pred is: ', Y_pred)
  f1 = f1_score(Y, Y_pred)
  #print('f1 score: ', f1)
  #print('precision, recall, f1 score are (ignore None): ', precision_recall_fscore_support(Y, Y_pred, average = 'binary'))
  print(classification_report(Y, Y_pred))
  model.train()

In [ ]:
torch.save(model.state_dict(), '/home/neem/weights/weights_EfficientNetB0.pt')

In [ ]:
print('Checking accuracy on val set:')
check_accuracy(val_loader, model)

In [ ]:
print('Checking accuracy on test set:')
check_accuracy(test_loader, model)

In [ ]:
print('Checking accuracy on training set:')
check_accuracy(train_loader, model)

# Resnext_50


In [ ]:
!nvidia-smi

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import numpy as np
import random
from PIL import Image
from skimage import io
from skimage.color import rgb2gray

In [ ]:
class train(Dataset):
    def __init__(self, train_csv_file,root_dir, transform=None):
        self.train_annotations = pd.read_csv(train_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.train_annotations)

    def __getitem__(self, index):
        # load a regular image
        image_path = os.path.join(self.root_dir, self.train_annotations.iloc[index, 0])
        y_label = int(self.train_annotations.iloc[index, 1])

        image = io.imread(image_path)

        # for greyscale images
        if len(image.shape) == 2:
            # Duplicate single channel across all three channels
            image = np.repeat(image[:, :, np.newaxis], 3, axis=2)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)


class test(Dataset):
    def __init__(self, test_csv_file, root_dir, transform=None):
        self.test_annotations = pd.read_csv(test_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.test_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.test_annotations.iloc[index, 0])
        y_label = int(self.test_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

class val(Dataset):
    def __init__(self, val_csv_file, root_dir, transform=None):
        self.val_annotations = pd.read_csv(val_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.val_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.val_annotations.iloc[index, 0])
        y_label = int(self.val_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyperparameters
in_channel = 3
learning_rate = 1e-3
batch_size = 2
num_epochs = 100

#transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.Resize((640,640))
])

train_dataset = train(train_csv_file='/home/neem/a_train.csv', root_dir='/home/neem/a_dataset', transform=transform)
test_dataset = test(test_csv_file='/home/neem/a_test.csv', root_dir='/home/neem/a_dataset', transform=transform)
val_dataset = val(val_csv_file='/home/neem/a_val.csv', root_dir='/home/neem/a_dataset', transform=transform)

In [ ]:
print(len(train_dataset))

In [ ]:
print(len(test_dataset))

In [ ]:
print(len(val_dataset))

In [ ]:
#train_set, test_set, val_set = torch.utils.data.random_split(dataset, [2834, 354, 354], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for items in train_loader:
    print(items)

In [ ]:
len(train_loader)

In [ ]:
#model
model = torchvision.models.resnext50_32x4d(pretrained=True)
model.to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Train network
t_loss = []
v_loss = []
for epoch in range(num_epochs):
  losses = []
  model.train()
  for batch_idx, (data, targets) in enumerate(train_loader):
    #data to cuda if possible
    data = data.to(device = device)
    targets = targets.to(device = device)

    #forward
    scores = model(data)
    loss = criterion(scores, targets)
    losses.append(loss.item())

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss = sum(losses)/len(losses)
    #print(f'Training loss at epoch {epoch} is {train_loss}')

  model.eval()
  loss = 0
  with torch.no_grad():
    for x, y in val_loader:
      x, y = x.to(device=device), y.to(device=device)
      out = model(x)
      loss += criterion(out, y)
  val_loss = loss/len(val_loader)
  print(f'Training and Validation loss at epoch {epoch} is {train_loss} and {val_loss}')
  t_loss.append(train_loss)
  v_loss.append(val_loss.item())

In [ ]:
import matplotlib.pyplot as plt
plt1 = plt.figure(figsize=(25,15))

x_axis = []
for i in range(num_epochs):
  x_axis.append(i)
plt.plot(x_axis, t_loss, 'g', label = 'training loss')
plt.plot(x_axis, v_loss, 'r', label = 'validation loss')
plt.legend(['training loss', 'validation loss'])
plt.title('training loss vs validation loss for Resnext_50')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()
  Y = np.array([0])
  Y_pred = np.array([0])
  f1 = 0
  i=0
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)
      y = y.to(device)
      Y = np.append(Y,y.cpu())
      i += 1
      print('Y after appendingt ', i, ' times : ', Y)
      scores = model(x)
      _, predictions = scores.max(1)
      print(list(predictions))
      Y_pred = np.append(Y_pred, predictions.cpu())
      print('Y_pred after appending: ', Y_pred)
      num_correct += (predictions == y).sum()
      num_samples = predictions.size(0)
    print(f'got accuracy {float(num_correct)/float(num_samples*100)}')
  Y = torch.IntTensor(Y)
  print('Y is : ', Y)
  Y_pred = torch.IntTensor(Y_pred)
  print('Y_pred is: ', Y_pred)
  f1 = f1_score(Y, Y_pred)
  #print('f1 score: ', f1)
  #print('precision, recall, f1 score are (ignore None): ', precision_recall_fscore_support(Y, Y_pred, average = 'binary'))
  print(classification_report(Y, Y_pred))
  model.train()

In [ ]:
torch.save(model.state_dict(), '/home/neem/weights/weights_Resnext_50.pt')

In [ ]:
print('Checking accuracy on val set:')
check_accuracy(val_loader, model)

In [ ]:
print('Checking accuracy on test set:')
check_accuracy(test_loader, model)

In [ ]:
print('Checking accuracy on training set:')
check_accuracy(train_loader, model)

# Resnet18

In [ ]:
!nvidia-smi

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import numpy as np
import random
from PIL import Image
from skimage import io
from skimage.color import rgb2gray

In [ ]:
class train(Dataset):
    def __init__(self, train_csv_file,root_dir, transform=None):
        self.train_annotations = pd.read_csv(train_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.train_annotations)

    def __getitem__(self, index):
        # load a regular image
        image_path = os.path.join(self.root_dir, self.train_annotations.iloc[index, 0])
        y_label = int(self.train_annotations.iloc[index, 1])

        image = io.imread(image_path)

        # for greyscale images
        if len(image.shape) == 2:
            # Duplicate single channel across all three channels
            image = np.repeat(image[:, :, np.newaxis], 3, axis=2)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)


class test(Dataset):
    def __init__(self, test_csv_file, root_dir, transform=None):
        self.test_annotations = pd.read_csv(test_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.test_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.test_annotations.iloc[index, 0])
        y_label = int(self.test_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

class val(Dataset):
    def __init__(self, val_csv_file, root_dir, transform=None):
        self.val_annotations = pd.read_csv(val_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.val_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.val_annotations.iloc[index, 0])
        y_label = int(self.val_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyperparameters
in_channel = 3
learning_rate = 1e-3
batch_size = 2
num_epochs = 100

#transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.Resize((640,640))
])

train_dataset = train(train_csv_file='/home/neem/a_train.csv', root_dir='/home/neem/a_dataset', transform=transform)

test_dataset = test(test_csv_file='/home/neem/a_test.csv', root_dir='/home/neem/a_dataset', transform=transform)

val_dataset = val(val_csv_file='/home/neem/a_val.csv', root_dir='/home/neem/a_dataset', transform=transform)

In [ ]:
print(len(train_dataset))

In [ ]:
print(len(test_dataset))

In [ ]:
print(len(val_dataset))

In [ ]:
#train_set, test_set, val_set = torch.utils.data.random_split(dataset, [2834, 354, 354], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for items in train_loader:
    print(items)

In [ ]:
len(train_loader)

In [ ]:
#model
model = torchvision.models.resnet18(pretrained=True)
model.to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Train network
t_loss = []
v_loss = []
for epoch in range(num_epochs):
  losses = []
  model.train()
  for batch_idx, (data, targets) in enumerate(train_loader):
    #data to cuda if possible
    data = data.to(device = device)
    targets = targets.to(device = device)

    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    losses.append(loss.item())

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss = sum(losses)/len(losses)
  #print(f'Training loss at epoch {epoch} is {train_loss}')

  model.eval()
  loss = 0
  with torch.no_grad():
    for x, y in val_loader:
      x, y = x.to(device=device), y.to(device=device)reached
      out = model(x)
      loss += criterion(out, y)
  val_loss = loss/len(val_loader)
  print(f'Training and Validation loss at epoch {epoch} is {train_loss} and {val_loss}')
  t_loss.append(train_loss)
  v_loss.append(val_loss.item())

In [ ]:
import matplotlib.pyplot as plt
plt1 = plt.figure(figsize=(25,15))

x_axis = []
for i in range(num_epochs):
  x_axis.append(i)
plt.plot(x_axis, t_loss, 'g', label = 'training loss')
plt.plot(x_axis, v_loss, 'r', label = 'validation loss')
plt.legend(['training loss', 'validation loss'])
plt.title('training loss vs validation loss for Resnet18')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()
  Y = np.array([0])
  Y_pred = np.array([0])
  f1 = 0
  i=0
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)
      y = y.to(device)
      Y = np.append(Y,y.cpu())
      i += 1
      print('Y after appendingt ', i, ' times : ', Y)
      scores = model(x)
      _, predictions = scores.max(1)
      print(list(predictions))
      Y_pred = np.append(Y_pred, predictions.cpu())
      print('Y_pred after appending: ', Y_pred)
      num_correct += (predictions == y).sum()
      num_samples = predictions.size(0)
    print(f'got accuracy {float(num_correct)/float(num_samples*100)}')
  Y = torch.IntTensor(Y)
  print('Y is : ', Y)
  Y_pred = torch.IntTensor(Y_pred)
  print('Y_pred is: ', Y_pred)
  f1 = f1_score(Y, Y_pred)
  #print('f1 score: ', f1)
  #print('precision, recall, f1 score are (ignore None): ', precision_recall_fscore_support(Y, Y_pred, average = 'binary'))
  print(classification_report(Y, Y_pred))
  model.train()

In [ ]:
torch.save(model.state_dict(), '/home/neem/weights/weights_Resnet18.pt')

In [ ]:
print('Checking accuracy on val set:')
check_accuracy(val_loader, model)

In [ ]:
print('Checking accuracy on test set:')
check_accuracy(test_loader, model)

In [ ]:
print('Checking accuracy on training set:')
check_accuracy(train_loader, model)

# SqueezeNet

In [ ]:
!nvidia-smi

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
import numpy as np
import random
from PIL import Image
from skimage import io
from skimage.color import rgb2gray

In [ ]:
class train(Dataset):
    def __init__(self, train_csv_file,root_dir, transform=None):
        self.train_annotations = pd.read_csv(train_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.train_annotations)

    def __getitem__(self, index):
        # load a regular image
        image_path = os.path.join(self.root_dir, self.train_annotations.iloc[index, 0])
        y_label = int(self.train_annotations.iloc[index, 1])

        image = io.imread(image_path)

        # for greyscale images
        if len(image.shape) == 2:
            # Duplicate single channel across all three channels
            image = np.repeat(image[:, :, np.newaxis], 3, axis=2)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)


class test(Dataset):
    def __init__(self, test_csv_file, root_dir, transform=None):
        self.test_annotations = pd.read_csv(test_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.test_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.test_annotations.iloc[index, 0])
        y_label = int(self.test_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

class val(Dataset):
    def __init__(self, val_csv_file, root_dir, transform=None):
        self.val_annotations = pd.read_csv(val_csv_file)
        self.root_dir = root_dir
        self.dataset = Dataset
        self.transform = transform

    def __len__(self):
        return len(self.val_annotations)

    def __getitem__(self, index):

        # load a regular image
        image_path = os.path.join(self.root_dir, self.val_annotations.iloc[index, 0])
        y_label = int(self.val_annotations.iloc[index, 1])

        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)
        return (image, y_label)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyperparameters
in_channel = 3
learning_rate = 1e-3
batch_size = 2
num_epochs = 100

#transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.Resize((640,640))
])

train_dataset = train(train_csv_file='/home/neem/a_train.csv', root_dir='/home/neem/a_dataset', transform=transform)

test_dataset = test(test_csv_file='/home/neem/a_test.csv', root_dir='/home/neem/a_dataset', transform=transform)

val_dataset = val(val_csv_file='/home/neem/a_val.csv', root_dir='/home/neem/a_dataset', transform=transform)

In [ ]:
print(len(train_dataset))

In [ ]:
print(len(test_dataset))

In [ ]:
print(len(val_dataset))

In [ ]:
#train_set, test_set, val_set = torch.utils.data.random_split(dataset, [2834, 354, 354], generator=torch.Generator().manual_seed(42))
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for items in train_loader:
    print(items)

In [ ]:
len(train_loader)

In [ ]:
#model
model = torchvision.models.squeezenet1_0(pretrained=True)
model.to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Train network
t_loss = []
v_loss = []
for epoch in range(num_epochs):
  losses = []
  model.train()
  for batch_idx, (data, targets) in enumerate(train_loader):
    #data to cuda if possible
    data = data.to(device = device)
    targets = targets.to(device = device)

    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    losses.append(loss.item())

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  train_loss = sum(losses)/len(losses)
  #print(f'Training loss at epoch {epoch} is {train_loss}')

  model.eval()
  loss = 0
  with torch.no_grad():
    for x, y in val_loader:
      x, y = x.to(device=device), y.to(device=device)
      out = model(x)
      loss += criterion(out, y)
  val_loss = loss/len(val_loader)
  print(f'Training and Validation loss at epoch {epoch} is {train_loss} and {val_loss}')
  t_loss.append(train_loss)
  v_loss.append(val_loss.item())

In [ ]:
import matplotlib.pyplot as plt
plt1 = plt.figure(figsize=(25,15))

x_axis = []
for i in range(num_epochs):
  x_axis.append(i)
plt.plot(x_axis, t_loss, 'g', label = 'training loss')
plt.plot(x_axis, v_loss, 'r', label = 'validation loss')
plt.legend(['training loss', 'validation loss'])
plt.title('training loss vs validation loss for SqueezeNet')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()
  Y = np.array([0])
  Y_pred = np.array([0])
  f1 = 0
  i=0
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)
      y = y.to(device)
      Y = np.append(Y,y.cpu())
      i += 1
      print('Y after appendingt ', i, ' times : ', Y)
      scores = model(x)
      _, predictions = scores.max(1)
      print(list(predictions))
      Y_pred = np.append(Y_pred, predictions.cpu())
      print('Y_pred after appending: ', Y_pred)
      num_correct += (predictions == y).sum()
      num_samples = predictions.size(0)
    print(f'got accuracy {float(num_correct)/float(num_samples*100)}')
  Y = torch.IntTensor(Y)
  print('Y is : ', Y)
  Y_pred = torch.IntTensor(Y_pred)
  print('Y_pred is: ', Y_pred)
  f1 = f1_score(Y, Y_pred)
  #print('f1 score: ', f1)
  #print('precision, recall, f1 score are (ignore None): ', precision_recall_fscore_support(Y, Y_pred, average = 'binary'))
  print(classification_report(Y, Y_pred))
  model.train()

In [ ]:
torch.save(model.state_dict(), '/home/neem/weights/weights_SqueezeNet.pt')

In [ ]:
print('Checking accuracy on val set:')
check_accuracy(val_loader, model)

In [ ]:
print('Checking accuracy on test set:')
check_accuracy(test_loader, model)

In [ ]:
print('Checking accuracy on training set:')
check_accuracy(train_loader, model)